Greatest common divisor/highest common factor:
    GCD := (λgmn. LEQ m n (g n m) (g m n)) (Y (λgxy. ISZERO y x (g y (MOD x y))))

[Details about algorithm](https://jwodder.freeshell.org/lambda.html)

In [1]:
import time

from lambda_calculus import Variable as Var
from lambda_calculus import Abstraction as Lambda
from lambda_calculus import Application as App

from lambda_calculus.visitors.normalisation import BetaNormalisingVisitor

from lambda_calculus.terms import logic
from lambda_calculus.terms import arithmetic
from lambda_calculus.terms import pairs
from lambda_calculus.terms import combinators

In [2]:
def multi_app_term(term_0, term_1, *terms):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term

Less than:
LT := λab. NOT (LEQ b a)

Division — DIV a b evaluates to a pair of two numbers, a idiv b and a mod b:
DIV := Y (λgqab. LT a b (PAIR q a) (g (SUCC q) (SUB a b) b) ) 0

CDR p — get the cdr of pair p; also called SECOND, TAIL, or REST:
SECOND := CDR := λp. p FALSE

Modulus:
MOD := λab. CDR (DIV a b)

In [3]:
def leq_term():
    m, n = "mm", "nn"
    m_, n_ = Var(m), Var(n)
    return Lambda(m, Lambda(n, App(
        arithmetic.ISZERO,
        App(App(arithmetic.SUBTRACT, m_), n_)
    )))


def lt_term():
    a, b = "aa", "bb"
    a_, b_ = Var(a), Var(b)
    return Lambda(a, Lambda(b, App(logic.NOT, multi_app_term(leq_term(), b_, a_))))


def div_term():
    g, q, a, b = "ggg", "qqq", "aaa", "bbb"
    g_, q_, a_, b_ = Var(g), Var(q), Var(a), Var(b)

    inner_right_term = multi_app_term(g_, App(arithmetic.SUCCESSOR, q_), multi_app_term(arithmetic.SUBTRACT, a_, b_), b_)
    inner_lambda_term = Lambda(g, Lambda(q, Lambda(a, Lambda(b, multi_app_term(lt_term(), a_, b_, multi_app_term(pairs.PAIR, q_, a_))))))

    return multi_app_term(combinators.Y, inner_lambda_term, inner_right_term, arithmetic.number(0))


def mod_term():
    a, b = "aaaa", "bbbb"
    a_, b_ = Var(a), Var(b)
    return Lambda(a, Lambda(b, App(pairs.SECOND, multi_app_term(div_term(), a_, b_))))

Greatest common divisor/highest common factor:
    GCD := (λgmn. LEQ m n (g n m) (g m n)) (Y (λgxy. ISZERO y x (g y (MOD x y))))

In [4]:
def gcd_term_v3():
    g, m, n, x, y = "g", "m", "n", "x", "y"
    g_, m_, n_, x_, y_ = Var(g), Var(m), Var(n), Var(x), Var(y)

    left_inner_term = Lambda(g, Lambda(m, Lambda(n, multi_app_term(
        leq_term(), m_, n_,
        multi_app_term(g_, n_, m_),
        multi_app_term(g_, m_, n_)
    ))))

    right_inner_term = Lambda(g, Lambda(x, Lambda(y, multi_app_term(
        arithmetic.ISZERO, y_, x_,
        multi_app_term(g_, y_, multi_app_term(mod_term(), x_, y_))
    ))))
    right_inner_term = App(combinators.Y, right_inner_term)

    return App(left_inner_term, right_inner_term)

In [5]:
def test_gcd_v3(var_0=0, var_1=0):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v3(), arithmetic.number(var_0), arithmetic.number(var_1))
    res_term = BetaNormalisingVisitor().skip_intermediate(term_)

    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [6]:
test_gcd_v3(0, 0)

(λf.(λx.x))
time normalization: 0.0010006427764892578s


In [7]:
test_gcd_v3(1, 0)

(λf.(λx.(f x)))
time normalization: 0.003000020980834961s


In [8]:
test_gcd_v3(0, 1)

(λf.(λx.(f x)))
time normalization: 0.0020003318786621094s


In [9]:
test_gcd_v3(1, 1)

(λf.(λx.(f x)))
time normalization: 0.004001140594482422s


In [10]:
test_gcd_v3(2, 2)

(λf.(λx.(f (f x))))
time normalization: 0.0050008296966552734s


In [11]:
test_gcd_v3(4, 2)

(λf.(λx.(f (f x))))
time normalization: 0.004000186920166016s


In [12]:
test_gcd_v3(2, 4)

(λf.(λx.(f (f x))))
time normalization: 0.00400090217590332s


In [13]:
test_gcd_v3(6, 3)

(λf.(λx.(f (f (f x)))))
time normalization: 0.0040013790130615234s


In [14]:
test_gcd_v3(3, 6)

(λf.(λx.(f (f (f x)))))
time normalization: 0.004000663757324219s


In [15]:
test_gcd_v3(10, 0)

(λf.(λx.(f (f (f (f (f (f (f (f (f (f x))))))))))))
time normalization: 0.003000497817993164s


In [16]:
test_gcd_v3(0, 10)

(λf.(λx.(f (f (f (f (f (f (f (f (f (f x))))))))))))
time normalization: 0.0030002593994140625s


In [17]:
test_gcd_v3(100, 10)

(λf.(λx.(f (f (f (f (f (f (f (f (f (f x))))))))))))
time normalization: 0.0710148811340332s


In [18]:
test_gcd_v3(10, 100)

(λf.(λx.(f (f (f (f (f (f (f (f (f (f x))))))))))))
time normalization: 0.2540571689605713s
